In [2]:
#importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Implementing KNN model

#### preparing Data

In [3]:
#this is the path of the dataset
path = '/Users/syedfaquaruddinquadri/Documents/Applied ML and DM/Final Project/machine-learning-dse-i210-final-project-yelprestaurantrecommender/data/processed'

In [4]:
#reading the business data into pandas data frame
nola_business_df = pd.read_csv(path+"/business.csv")

In [5]:
#peeking into the dataset
nola_business_df.head()

,Unnamed: 0,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,0,712 Adams St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",M0XSSHqrASOnhgbWDJIpQA,"Vape Shops, Tobacco Shops, Personal Shopping, ...",New Orleans,"Row(Friday='10:0-19:0', Monday='10:0-19:0', Sa...",1,29.941468,-90.129953,Herb Import Co,70118.0,5,4.0,LA
1,1,100 Iberville St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",w_AMNoI1iG9eay7ncmc67w,"Event Planning & Services, Hotels, Hotels & Tr...",New Orleans,NaN,1,29.951359,-90.064672,River 127,70130.0,12,3.0,LA
2,2,723 St Peter St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",uczmbBk5O3tYhGue13dCDg,"Hotels & Travel, Tours, Local Flavor",New Orleans,"Row(Friday='10:0-20:0', Monday='0:0-0:0', Satu...",1,29.958431,-90.065173,New Orleans Spirit Tours,70130.0,38,4.0,LA
3,3,1001 Poydras St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",YNjyv0gfOr2g8lbmUpTnKg,"Nightlife, Pubs, Event Planning & Services, Wi...",New Orleans,"Row(Friday='11:0-23:0', Monday='0:0-0:0', Satu...",1,29.950647,-90.074427,Copper Vine,70112.0,350,4.5,LA
4,4,600 Decatur St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",J_ksUDPpzPwfTGtI4zTRnQ,"Event Planning & Services, Caterers, Party & E...",New Orleans,"Row(Friday='9:0-16:0', Monday='9:0-16:0', Satu...",1,29.955925,-90.062962,Riverview Room,70130.0,7,4.5,LA


In [6]:
#checking the columns if the dataframe
nola_business_df.columns

Index(['Unnamed: 0', 'address', 'attributes', 'business_id', 'categories',
       'city', 'hours', 'is_open', 'latitude', 'longitude', 'name',
       'postal_code', 'review_count', 'stars', 'state'],
      dtype='object')

In [7]:
nola_business_df.shape

(6217, 15)

In [8]:
nola_business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6217 entries, 0 to 6216
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    6217 non-null   int64  
 1   address       5954 non-null   object 
 2   attributes    5722 non-null   object 
 3   business_id   6217 non-null   object 
 4   categories    6216 non-null   object 
 5   city          6217 non-null   object 
 6   hours         5059 non-null   object 
 7   is_open       6217 non-null   int64  
 8   latitude      6217 non-null   float64
 9   longitude     6217 non-null   float64
 10  name          6217 non-null   object 
 11  postal_code   6212 non-null   float64
 12  review_count  6217 non-null   int64  
 13  stars         6217 non-null   float64
 14  state         6217 non-null   object 
dtypes: float64(4), int64(3), object(8)
memory usage: 728.7+ KB


In [9]:
nola_business_df.describe()

,Unnamed: 0,is_open,latitude,longitude,postal_code,review_count,stars
count,6217.000000,6217.000000,6217.000000,6217.000000,6212.000000,6217.000000,6217.000000
mean,3108.000000,0.748914,29.956551,-90.079817,70123.672408,99.976516,3.822825
std,1794.837644,0.433673,0.025970,0.036654,158.234691,314.793646,0.873893
min,0.000000,0.000000,29.784183,-90.260194,70001.000000,5.000000,1.000000
25%,1554.000000,0.000000,29.941588,-90.099613,70115.000000,9.000000,3.500000
50%,3108.000000,1.000000,29.954678,-90.073135,70119.000000,22.000000,4.000000
75%,4662.000000,1.000000,29.964969,-90.064741,70130.000000,67.000000,4.500000
max,6216.000000,1.000000,30.133632,-89.866514,79115.000000,7568.000000,5.000000


In [10]:
#dropping the columns we dont need
nola_business_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [11]:
nola_business_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,712 Adams St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",M0XSSHqrASOnhgbWDJIpQA,"Vape Shops, Tobacco Shops, Personal Shopping, ...",New Orleans,"Row(Friday='10:0-19:0', Monday='10:0-19:0', Sa...",1,29.941468,-90.129953,Herb Import Co,70118.0,5,4.0,LA
1,100 Iberville St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",w_AMNoI1iG9eay7ncmc67w,"Event Planning & Services, Hotels, Hotels & Tr...",New Orleans,NaN,1,29.951359,-90.064672,River 127,70130.0,12,3.0,LA
2,723 St Peter St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",uczmbBk5O3tYhGue13dCDg,"Hotels & Travel, Tours, Local Flavor",New Orleans,"Row(Friday='10:0-20:0', Monday='0:0-0:0', Satu...",1,29.958431,-90.065173,New Orleans Spirit Tours,70130.0,38,4.0,LA
3,1001 Poydras St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",YNjyv0gfOr2g8lbmUpTnKg,"Nightlife, Pubs, Event Planning & Services, Wi...",New Orleans,"Row(Friday='11:0-23:0', Monday='0:0-0:0', Satu...",1,29.950647,-90.074427,Copper Vine,70112.0,350,4.5,LA
4,600 Decatur St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",J_ksUDPpzPwfTGtI4zTRnQ,"Event Planning & Services, Caterers, Party & E...",New Orleans,"Row(Friday='9:0-16:0', Monday='9:0-16:0', Satu...",1,29.955925,-90.062962,Riverview Room,70130.0,7,4.5,LA


In [12]:
#checking the different spelling of the city 
nola_business_df.city.unique()

array(['New Orleans', 'NEW ORLEANS', 'New orleans', 'New Orlaens',
       'new orleans'], dtype=object)

In [13]:
#checking the state column
nola_business_df.state.unique()

array(['LA', 'FL'], dtype=object)

new orleans is in LA but FL shows in the uniques values, checking further the number of the misleading data points

In [14]:
nola_business_df[nola_business_df['state'] ==  'FL'].shape

(1, 14)

there is only one row in which state is florida, one data point will not have a drastic effect on our analysis so i am gonna leave it as it is

In [15]:
nola_business_df.categories.unique()

array(['Vape Shops, Tobacco Shops, Personal Shopping, Vitamins & Supplements, Shopping',
       'Event Planning & Services, Hotels, Hotels & Travel',
       'Hotels & Travel, Tours, Local Flavor', ...,
       'Vietnamese, Restaurants, Chinese, Asian Fusion',
       'Shopping, Home & Garden, Home Decor, Home Services, Paint Stores, Painters, Contractors',
       'Shopping, Food, Grocery'], dtype=object)

In [16]:
rest_business = nola_business_df[nola_business_df['categories'].str.contains('Restaurants')==True]

In [17]:
rest_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
3,1001 Poydras St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",YNjyv0gfOr2g8lbmUpTnKg,"Nightlife, Pubs, Event Planning & Services, Wi...",New Orleans,"Row(Friday='11:0-23:0', Monday='0:0-0:0', Satu...",1,29.950647,-90.074427,Copper Vine,70112.0,350,4.5,LA
5,901 Iberville St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",TLZ3-eDPLhUzfsWO4ad6Ug,"Restaurants, Seafood, Cajun/Creole",New Orleans,"Row(Friday='12:0-20:0', Monday='0:0-0:0', Satu...",1,29.955415,-90.070062,Mahony's Po-Boys & Seafood,70112.0,382,4.0,LA
6,2127 Prytania St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",FRYkg_JvsWU9xIXZsEZcVA,"Cocktail Bars, Italian, Nightlife, Seafood, Ba...",New Orleans,"Row(Friday='17:0-23:0', Monday='17:0-22:0', Sa...",0,29.933388,-90.079498,Altamura,70115.0,27,3.5,LA
8,949 N Rendon St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",4IcB3QyMEA85UTWFKh9O9A,"American (New), Food, Bars, Nightlife, Lounges...",New Orleans,"Row(Friday=None, Monday=None, Saturday=None, S...",0,29.976762,-90.086423,Eat Mah Taco @ Pal's Lounge,70119.0,8,4.5,LA
9,8227 Oak St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",Edg22x3CZkIv0GUib2oEFA,"Pizza, Restaurants, Bars, Nightlife, Sandwiches",New Orleans,"Row(Friday='11:0-23:0', Monday='0:0-0:0', Satu...",0,29.948624,-90.131250,Mellow Mushroom,70118.0,149,3.5,LA


In [18]:
rest_business.shape

(2261, 14)

In [19]:
rest_business.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'stars', 'state'],
      dtype='object')

In [20]:
#comtains only features that are required
nola_rest = rest_business[['attributes', 'categories', 'business_id', 'review_count', 'stars', 'name']]

In [21]:
nola_rest.head()

,attributes,categories,business_id,review_count,stars,name
3,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine
5,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Restaurants, Seafood, Cajun/Creole",TLZ3-eDPLhUzfsWO4ad6Ug,382,4.0,Mahony's Po-Boys & Seafood
6,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Cocktail Bars, Italian, Nightlife, Seafood, Ba...",FRYkg_JvsWU9xIXZsEZcVA,27,3.5,Altamura
8,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","American (New), Food, Bars, Nightlife, Lounges...",4IcB3QyMEA85UTWFKh9O9A,8,4.5,Eat Mah Taco @ Pal's Lounge
9,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Pizza, Restaurants, Bars, Nightlife, Sandwiches",Edg22x3CZkIv0GUib2oEFA,149,3.5,Mellow Mushroom


There are 2261 restaurants in the new orleans city i have verified this number with EDA notebook 

In [22]:
#reading the reviews csv into dataframe
reviews_path = '/Users/syedfaquaruddinquadri/Documents/Applied ML and DM/Final Project'
reviews = pd.read_csv(reviews_path+"/reviews.csv")

/var/folders/6y/65h__9y57mg1xqznxbt3fvtr0000gn/T/ipykernel_24372/869891069.py:3: DtypeWarning: Columns (0,1,2,3,5,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.read_csv(reviews_path+"/reviews.csv")


In [23]:
reviews.head()

,Unnamed: 0,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0,e4Vwtrqf-wpJfwesgvdgxQ,1,2017-01-14 20:54:15,0.0,Sx8TMOWLNuJBWer-0pcmoA,4.0,Cute interior and owner (?) gave us tour of up...,1.0,bcjbaE6dDog4jkNY91ncLQ
1,1,gmjsEdUsKpj9Xxu6pdjH0g,0,2015-01-03 23:21:18,2.0,6AxgBCNX_PNTOxmbRSwcKQ,5.0,Loved this tour! I grabbed a groupon and the p...,0.0,r3zeYsv1XFBRA4dJpL78cw
2,2,S2Ho8yLxhKAa26pBAm6rxA,0,2016-11-22 00:22:53,0.0,z0osLHDvXvzfm57D4DmD2Q,3.0,"Service was crappy, and food was mediocre. I ...",0.0,xVKE_HJ2pwUtTdLbL3pnCg
3,3,W4ZEKkva9HpAdZG88juwyQ,0,2013-12-29 02:37:42,0.0,jC-fGfx-YLqxVBcyTAd4Pw,3.0,"In a word... ""OVERRATED!"". The food took fore...",0.0,EBa-0-6AKoy6jziNexDJtg
4,4,j8JOZvfeHEfUWq3gEz6ABQ,0,2014-06-11 14:55:14,3.0,Ki90-hauSIfW_7sBuBBqfA,2.0,NEVER AGAIN. This is a so called restaurant th...,2.0,Z2cOL3n9V8NoguJ-uU_Nvw


In [24]:
reviews.shape

(1095800, 10)

In [25]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095800 entries, 0 to 1095799
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Unnamed: 0   1095800 non-null  object 
 1   business_id  702584 non-null   object 
 2   cool         702584 non-null   object 
 3   date         702582 non-null   object 
 4   funny        702582 non-null   float64
 5   review_id    702582 non-null   object 
 6   stars        702582 non-null   float64
 7   text         702582 non-null   object 
 8   useful       635559 non-null   float64
 9   user_id      635559 non-null   object 
dtypes: float64(3), object(7)
memory usage: 83.6+ MB


In [26]:
reviews.describe()

,funny,stars,useful
count,702582.000000,702582.000000,635559.000000
mean,0.496167,3.695177,1.069893
std,1.556895,1.506961,2.683978
min,0.000000,1.000000,0.000000
25%,0.000000,3.000000,0.000000
50%,0.000000,4.000000,0.000000
75%,0.000000,5.000000,1.000000
max,201.000000,5.000000,262.000000


In [27]:
reviews.columns

Index(['Unnamed: 0', 'business_id', 'cool', 'date', 'funny', 'review_id',
       'stars', 'text', 'useful', 'user_id'],
      dtype='object')

In [28]:
reviews.drop(['Unnamed: 0', 'cool', 'date', 'funny', 'useful'], 
             axis=1, 
             inplace=True)

In [29]:
reviews.columns

Index(['business_id', 'review_id', 'stars', 'text', 'user_id'], dtype='object')

In [30]:
reviews.head()

,business_id,review_id,stars,text,user_id
0,e4Vwtrqf-wpJfwesgvdgxQ,Sx8TMOWLNuJBWer-0pcmoA,4.0,Cute interior and owner (?) gave us tour of up...,bcjbaE6dDog4jkNY91ncLQ
1,gmjsEdUsKpj9Xxu6pdjH0g,6AxgBCNX_PNTOxmbRSwcKQ,5.0,Loved this tour! I grabbed a groupon and the p...,r3zeYsv1XFBRA4dJpL78cw
2,S2Ho8yLxhKAa26pBAm6rxA,z0osLHDvXvzfm57D4DmD2Q,3.0,"Service was crappy, and food was mediocre. I ...",xVKE_HJ2pwUtTdLbL3pnCg
3,W4ZEKkva9HpAdZG88juwyQ,jC-fGfx-YLqxVBcyTAd4Pw,3.0,"In a word... ""OVERRATED!"". The food took fore...",EBa-0-6AKoy6jziNexDJtg
4,j8JOZvfeHEfUWq3gEz6ABQ,Ki90-hauSIfW_7sBuBBqfA,2.0,NEVER AGAIN. This is a so called restaurant th...,Z2cOL3n9V8NoguJ-uU_Nvw


In [31]:
rev = pd.merge(nola_rest, 
         reviews,left_on='business_id', 
         right_on='business_id', how = 'inner')

In [32]:
rev.head()

,attributes,categories,business_id,review_count,stars_x,name,review_id,stars_y,text,user_id
0,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine,2u8kIWm1CrMGuwGTW1HBGQ,5.0,The service is awesome...staff is very friendl...,nn6DoANEtr7SgvWWgrh2oQ
1,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine,EwarwhOOmnB22qESUv_VPw,5.0,New and cool spot in downtown New Orleans! Gre...,G6ZnatT96yzdcX81PZyT3g
2,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine,EdsUPECvSzIj503qt13Pwg,4.0,We went here before a concert. It was super bu...,dRvKAgf9a0DSKioJSv1p0Q
3,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine,XEfHuaszNLgeUu-6gXB-qQ,4.0,My friends and I had a great meal at Copper Vi...,JDOeSXX33nUx4q-AmUFBSw
4,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine,W7_4Dd6xTuWuTDqe68XBtQ,5.0,"What a great concept! 38 wines on tap, oh yeah...",GCldu8eAzez5rSFpijNZ3A


In [33]:
#renaming the columns
rev.rename(columns = {'stars_x':'rest_rating', 'stars_y':'user_rating', 'text':'text_review'}, inplace = True)

In [34]:
rev.head()

,attributes,categories,business_id,review_count,rest_rating,name,review_id,user_rating,text_review,user_id
0,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine,2u8kIWm1CrMGuwGTW1HBGQ,5.0,The service is awesome...staff is very friendl...,nn6DoANEtr7SgvWWgrh2oQ
1,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine,EwarwhOOmnB22qESUv_VPw,5.0,New and cool spot in downtown New Orleans! Gre...,G6ZnatT96yzdcX81PZyT3g
2,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine,EdsUPECvSzIj503qt13Pwg,4.0,We went here before a concert. It was super bu...,dRvKAgf9a0DSKioJSv1p0Q
3,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine,XEfHuaszNLgeUu-6gXB-qQ,4.0,My friends and I had a great meal at Copper Vi...,JDOeSXX33nUx4q-AmUFBSw
4,"Row(AcceptsInsurance=None, AgesAllowed=None, A...","Nightlife, Pubs, Event Planning & Services, Wi...",YNjyv0gfOr2g8lbmUpTnKg,350,4.5,Copper Vine,W7_4Dd6xTuWuTDqe68XBtQ,5.0,"What a great concept! 38 wines on tap, oh yeah...",GCldu8eAzez5rSFpijNZ3A


In [35]:
rev.user_id.unique().shape

(201870,)

we have 201870 unique users

In [36]:
rev.columns

Index(['attributes', 'categories', 'business_id', 'review_count',
       'rest_rating', 'name', 'review_id', 'user_rating', 'text_review',
       'user_id'],
      dtype='object')

In [37]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [40]:
%%time
# Creating an item-user interaction matrix
interaction_matrix = rev.pivot_table(index='business_id', columns='user_id', values='rest_rating', fill_value=0)


CPU times: user 22.3 s, sys: 14.9 s, total: 37.3 s
Wall time: 44.9 s


In [41]:
interaction_matrix

user_id,---UgP94gokyCDuB5zUssA,--1orhUoGFSdHXsoxqQc8g,--3PldvWEZ_bhLNsyrAtgA,--3WaS23LcIXtxyFULJHTA,--48iehfEvtnSrLq8grJIQ,--4wsx_CrQyZxJYvHjUuaw,--5IPONUyyQt38e85P1KdQ,--5VfF9MeCYnNiGFp3BXpw,--78aksX3obHJ6671pjJqA,--CIuK7sUpaNzalLAlHJKA,...,zzgLgszu48af4NJULymzAA,zzgY5GgMSc7Iin2W9oqd6w,zziWJMYwDjyVi7kJmgRUvg,zzisaH1aDfNlCTenS5co9w,zzlZ2oy-hOGXQGqDMF0IFg,zzq6Nhcq0JMbm7ExFFCP2A,zzqK0h_3FLhcKbX5H6rDZA,zzsqjDvanJhH9tn8NautOQ,zzuIXWg1WRhQXheMIdGZ5w,zzxZoMmjbUjXcWZzrE3PIw
business_id,,,,,,,,,,,,,,,,,,,,,
-0__F9fnKt8uioCKztF5Ww,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
-0ltw8--HLuulPyOSspqAQ,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
-1XSzguS6XLN-V6MVZMg2A,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
-4x3pVUUsfWmKEilWKsOZQ,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
-8B29WNRCyg4jViK4ChKlQ,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zvGNZF827KyzLupKiG4Xtw,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
zwe9H6Xxqb1_E09A20Ptgg,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0
zxfqXVgZ_NvLY2lc1DAlJQ,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0,...,0.0,0,0.0,0,0,0.0,0,0.0,0.0,0


In [42]:
%%time
# Converting the interaction matrix to a sparse matrix
sparse_matrix = csr_matrix(interaction_matrix.values)

CPU times: user 2min 20s, sys: 16min 51s, total: 19min 12s
Wall time: 27min 16s


In [43]:
print(sparse_matrix)

  (0, 3323)	4.0
  (0, 4147)	4.0
  (0, 4261)	4.0
  (0, 7182)	4.0
  (0, 7237)	4.0
  (0, 7359)	4.0
  (0, 12843)	4.0
  (0, 18939)	4.0
  (0, 19301)	4.0
  (0, 19327)	4.0
  (0, 27696)	4.0
  (0, 31372)	4.0
  (0, 35091)	4.0
  (0, 37693)	4.0
  (0, 38312)	4.0
  (0, 41974)	4.0
  (0, 43718)	4.0
  (0, 44494)	4.0
  (0, 45995)	4.0
  (0, 53515)	4.0
  (0, 54509)	4.0
  (0, 62887)	4.0
  (0, 64272)	4.0
  (0, 69811)	4.0
  (0, 73308)	4.0
  :	:
  (2259, 160732)	3.0
  (2259, 162495)	3.0
  (2259, 164268)	3.0
  (2259, 169205)	3.0
  (2259, 174785)	3.0
  (2259, 176401)	3.0
  (2259, 178158)	3.0
  (2259, 178736)	3.0
  (2259, 189372)	3.0
  (2259, 190577)	3.0
  (2259, 193967)	3.0
  (2259, 194991)	3.0
  (2259, 198617)	3.0
  (2260, 8843)	5.0
  (2260, 10628)	5.0
  (2260, 59039)	5.0
  (2260, 62271)	5.0
  (2260, 96729)	5.0
  (2260, 119720)	5.0
  (2260, 128200)	5.0
  (2260, 129638)	5.0
  (2260, 160095)	5.0
  (2260, 164243)	5.0
  (2260, 174354)	5.0
  (2260, 200161)	5.0


In [44]:
from sklearn.model_selection import train_test_split
# Split the data into a train and test set
train_data, test_data = train_test_split(rev[['user_id', 'business_id', 'rest_rating']], test_size=0.2, random_state=42)

In [47]:
%%time
# Create and fit the kNN model
k = 5
model = NearestNeighbors(n_neighbors=k, metric='cosine', algorithm='brute')
model.fit(sparse_matrix)

CPU times: user 4.24 ms, sys: 9.11 ms, total: 13.3 ms
Wall time: 15.4 ms


NearestNeighbors(algorithm='brute', metric='cosine')

In [48]:
user_id = 'nn6DoANEtr7SgvWWgrh2oQ'
user_idx = interaction_matrix.columns.get_loc(user_id)

# Getting the rating for each business the user has reviewed
user_ratings = interaction_matrix.iloc[:, user_idx]

#geting the mean rating for each business the user has reviewd
mean_user_rating = np.mean(user_ratings[user_ratings > 0])

In [49]:
# Predict ratings for all businesses using kNN
distances, indices = model.kneighbors(sparse_matrix, n_neighbors=k+1)
predicted_ratings = np.array([np.mean(interaction_matrix.iloc[indices[row_idx], user_idx]) 
                              for row_idx in range(interaction_matrix.shape[0])])

In [50]:
# Adjust the predicted ratings by adding the mean user rating
adjusted_ratings = predicted_ratings + mean_user_rating

In [51]:
# Get the indices of the top 5 recommended businesses
num_recommendations = 5
top_business_indices = adjusted_ratings.argsort()[-num_recommendations:][::-1]


In [52]:
# Get the business IDs of the top recommended businesses
top_business_ids = interaction_matrix.index[top_business_indices].tolist()

In [53]:
len(top_business_ids)

5

In [54]:
# selecting rows based on condition
recommended_restaurants = nola_rest.loc[nola_rest['business_id'].isin(top_business_ids)][['business_id', 'name']]


# dataframe.loc[dataframe['Stream'].isin(options)]

In [55]:
recommended_restaurants

,business_id,name
3,YNjyv0gfOr2g8lbmUpTnKg,Copper Vine
1284,el6K1DsZRnKNNaZ14YGMHQ,Salads Galore & More
1627,fUUNiG5x6wP84ty5GoqtRw,The Neutral Ground Bar & Grill
3451,UEPf0MGsw2mOlarskQX1Zg,Streetcar Poboys & Seafood
4881,ISm3ZCdmYsxwy7c8sBm1gw,Thaihey Nola


In [71]:
from sklearn.neighbors import NearestNeighbors
# Create and fit the kNN model
k = 5
model = NearestNeighbors(n_neighbors=k, 
                         metric='cosine', 
                         algorithm='brute')
model.fit(sparse_matrix)

# Function to make recommendations based on the nearest neighbors of an item
def recommend_restaurants(user_id, num_recommendations=5):
    user_idx = interaction_matrix.columns.get_loc(user_id)

    # Getting the rating for each business the user has reviewed
    user_ratings = interaction_matrix.iloc[:, user_idx]

    #geting the mean rating for each business the user has reviewd
    mean_user_rating = np.mean(user_ratings[user_ratings > 0])

    # Predict ratings for all businesses using kNN
    distances, indices = model.kneighbors(sparse_matrix, n_neighbors=k+1)
    predicted_ratings = np.array([np.mean(interaction_matrix.iloc[indices[row_idx], user_idx]) for row_idx in range(interaction_matrix.shape[0])])

    # Adjust the predicted ratings by adding the mean user rating
    adjusted_ratings = predicted_ratings + mean_user_rating

    # Get the indices of the top recommended businesses
    top_business_indices = adjusted_ratings.argsort()[-num_recommendations:][::-1]

    # Get the business IDs of the top recommended businesses
    top_business_ids = interaction_matrix.index[top_business_indices].tolist()
    
    recommended_restaurants = nola_rest.loc[nola_rest['business_id'].isin(top_business_ids)][['business_id', 'name']]


    return recommended_restaurants

# Example usage: Recommend restaurants for a specific user
user_id = 'nn6DoANEtr7SgvWWgrh2oQ'
recommended_restaurants = recommend_restaurants(user_id)
print(recommended_restaurants)




recommended restaurants:
******************
fUUNiG5x6wP84ty5GoqtRw
el6K1DsZRnKNNaZ14YGMHQ
UEPf0MGsw2mOlarskQX1Zg
ISm3ZCdmYsxwy7c8sBm1gw
YNjyv0gfOr2g8lbmUpTnKg


In [56]:

from sklearn.neighbors import NearestNeighbors

# Create and fit the kNN model
k = 5
model = NearestNeighbors(n_neighbors=k, 
                         metric='cosine', 
                         algorithm='brute')
model.fit(sparse_matrix)

# Function to make recommendations based on the nearest neighbors of an item
def recommend_restaurants(user_id, num_recommendations=5):
    user_idx = interaction_matrix.columns.get_loc(user_id)

    # Getting the rating for each business the user has reviewed
    user_ratings = interaction_matrix.iloc[:, user_idx]

    #geting the mean rating for each business the user has reviewd
    mean_user_rating = np.mean(user_ratings[user_ratings > 0])

    # Predict ratings for all businesses using kNN
    distances, indices = model.kneighbors(sparse_matrix, n_neighbors=k+1)
    predicted_ratings = np.array([np.mean(interaction_matrix.iloc[indices[row_idx], user_idx]) for row_idx in range(interaction_matrix.shape[0])])

    # Adjust the predicted ratings by adding the mean user rating
    adjusted_ratings = predicted_ratings + mean_user_rating

    # Get the indices of the top recommended businesses
    top_business_indices = adjusted_ratings.argsort()[-num_recommendations:][::-1]

    # Get the business IDs of the top recommended businesses
    top_business_ids = interaction_matrix.index[top_business_indices].tolist()
    
    recommended_restaurants = nola_rest.loc[nola_rest['business_id'].isin(top_business_ids)][['business_id', 'name']]


    return recommended_restaurants

# Example usage: Recommend restaurants for a specific user
user_id = 'nn6DoANEtr7SgvWWgrh2oQ'
recommended_restaurants = recommend_restaurants(user_id)
print(recommended_restaurants)


                 business_id                            name
3     YNjyv0gfOr2g8lbmUpTnKg                     Copper Vine
1284  el6K1DsZRnKNNaZ14YGMHQ            Salads Galore & More
1627  fUUNiG5x6wP84ty5GoqtRw  The Neutral Ground Bar & Grill
3451  UEPf0MGsw2mOlarskQX1Zg      Streetcar Poboys & Seafood
4881  ISm3ZCdmYsxwy7c8sBm1gw                    Thaihey Nola
